In [5]:
import pandas as pd
import numpy as np
import polars as pl
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import gzip
import shutil
import pathlib
import os
import sqlalchemy
import sqlite3
import spacy
import re
import spacy_cleaner
from spacy_cleaner import processing, Cleaner
import re
import tqdm
from tqdm.notebook import tqdm, trange
import ipywidgets as widgets
from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import time
import timeit
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor


from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()

%matplotlib inline
alt.data_transformers.disable_max_rows()


#spacy.prefer_gpu()
spacy.require_gpu(gpu_id=0)
nlp = spacy.load("en_core_web_sm")

# This loads a larger and more robust model. Use with caution though because it takes considerably longer to run
#nlp = spacy.load("en_core_web_trf")

#nlp = spacy.load('/path/to/en_core_web_sm')

In [2]:
def print_files_in_directory(directory_path):
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                print(entry.name)

def get_sentiment_score(text):
    sentiment_scores = sid_obj.polarity_scores(text)
    return sentiment_scores

def clean_text(text):
    doc = nlp(text)

    cleaned_tokens = [token.lemma_.lower().strip() for token in doc if not token.is_punct and not token.is_space]
    cleaned_tokens = [token for token in cleaned_tokens if not nlp.vocab[token].is_stop]
    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text

def remove_numbers(text):
    cleaner = Cleaner(
        nlp,
        processing.remove_number_token
    )
    return cleaner.clean(text)

def remove_numbers_regex(text):
    # Pattern to remove numbers from text data
    pattern = r"\d+"

    return re.sub(pattern,"",text)

In [14]:
conn = sqlite3.connect(r"C:/Users/asl4a/AirBnB_Data.db")
cursor = conn.cursor()

In [4]:
sql = """
SELECT
    reviews.id,
    reviews.temp_index,
    reviews.comments
FROM reviews"""

df_revs = pd.read_sql(sql,con=conn)

In [ ]:
df_revs.head()

In [ ]:

df_revs.info()

In [ ]:
sample_text = df_revs['comments'][:20]
sample_text

In [ ]:
sample_text = remove_numbers(df_revs['comments'][:20])
sample_text

In [ ]:
# Use the function that utilizes regex instead of the function with Spacy unless you want to wait 3 hours for the code to finish running
#df_revs['cleaned_text'] = remove_numbers(df_revs['comments'])
df_revs['cleaned_text'] = df_revs['comments'].apply(remove_numbers_regex)

In [ ]:
df_revs['comments'] = df_revs['comments'].astype(str)

In [ ]:
len(df_revs)

In [ ]:
#df_revs['cleaned_text'] = df_revs['cleaned_text'].apply(clean_text)
# This code has not yet run (as of 2/24/2024 14:46PM)
beg_slice = 8700000


total_iterations = len(df_revs['cleaned_text'][beg_slice:])
progress_bar = tqdm(total=total_iterations,desc='Processing text data...')

cleaned_text_list = []

for i in df_revs['cleaned_text'][beg_slice:]:
    cleaned_text_list.append(clean_text(i))
    progress_bar.update(1)

progress_bar.close()

In [14]:
cleaned_text_s = pd.Series(cleaned_text_list)
#cleaned_text_s.to_csv(r"F:\Data Science\Datasets\Cleaned AirBnB Reviews\cleaned_reviews_17.csv")

In [21]:
#df_revs['cleaned_text'] = cleaned_text_list

100000

In [7]:
cleaned_data_dir_path = r"F:\Data Science\Datasets\Cleaned AirBnB Reviews"

print_files_in_directory(cleaned_data_dir_path)

cleaned_reviews_1.csv
cleaned_reviews_10.csv
cleaned_reviews_11.csv
cleaned_reviews_12.csv
cleaned_reviews_13.csv
cleaned_reviews_14.csv
cleaned_reviews_15.csv
cleaned_reviews_17.csv
cleaned_reviews_2.csv
cleaned_reviews_3.csv
cleaned_reviews_4.csv
cleaned_reviews_5.csv
cleaned_reviews_6.csv
cleaned_reviews_7.csv
cleaned_reviews_8.csv
cleaned_reviews_9.csv


In [9]:
cleaned_reviews_list = []

with os.scandir(cleaned_data_dir_path) as entries:
    for entry in entries:
        if entry.is_file():
            cleaned_reviews_list.append(pd.read_csv(entry.path))

cleaned_revs_df = pd.concat(cleaned_reviews_list)
cleaned_revs_df.head()

,Unnamed: 0,0
0,0,wonderful stay highlight stay nashville michel...
1,1,michelle collier home wonderful great people m...
2,2,spend night michele home feel wonderful right ...
3,3,michele collier lovely people pleasure meet fa...
4,4,lovely time stay michele collier nashville hou...


In [10]:
cleaned_revs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9668045 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Unnamed: 0  int64 
 1   0           object
dtypes: int64(1), object(1)
memory usage: 221.3+ MB


In [17]:
cleaned_revs_df = cleaned_revs_df.rename(columns={'Unnamed: 0': 'temp_index','0': 'cleaned_reviews'})

In [19]:
cleaned_revs_df.to_sql(name='cleaned_reviews',con=conn,if_exists='replace')

9668045

In [20]:
sql = """
SELECT
    l.review_scores_rating,
    cr.cleaned_reviews
FROM
    listings l
JOIN cleaned_reviews cr ON l.index = cr.temp_index;
"""

cr_df = pd.read_sql(sql=sql,con=conn)

DatabaseError: Execution failed on sql '
SELECT
    l.review_scores_rating,
    cr.cleaned_reviews
FROM
    listings l
JOIN cleaned_reviews cr ON l.index = cr.temp_index;
': near "index": syntax error

In [24]:
sql = """
SELECT
    *
    --l."index",
    --l.review_scores_rating
FROM
    listings l;
"""
listings_df = pd.read_sql(sql=sql,con=conn)

In [25]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228701 entries, 0 to 228700
Data columns (total 76 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   index                                         228701 non-null  int64  
 1   id                                            228701 non-null  int64  
 2   listing_url                                   228701 non-null  object 
 3   scrape_id                                     228701 non-null  int64  
 4   last_scraped                                  228701 non-null  object 
 5   source                                        228701 non-null  object 
 6   name                                          228701 non-null  object 
 7   description                                   226441 non-null  object 
 8   neighborhood_overview                         144375 non-null  object 
 9   picture_url                                   22

In [26]:
listings_dest_path = r"F:\Data Science\Datasets\AirBnB Data\Listings"

print_files_in_directory(listings_dest_path)

listings (1).csv.gz
listings (10).csv.gz
listings (11).csv.gz
listings (12).csv.gz
listings (13).csv.gz
listings (14).csv.gz
listings (15).csv.gz
listings (16).csv.gz
listings (17).csv.gz
listings (18).csv.gz
listings (19).csv.gz
listings (2).csv.gz
listings (20).csv.gz
listings (21).csv.gz
listings (22).csv.gz
listings (23).csv.gz
listings (24).csv.gz
listings (25).csv.gz
listings (26).csv.gz
listings (27).csv.gz
listings (3).csv.gz
listings (4).csv.gz
listings (5).csv.gz
listings (6).csv.gz
listings (7).csv.gz
listings (8).csv.gz
listings (9).csv.gz
listings.csv.gz


In [28]:
gz_listings_path = r"F:\Data Science\Datasets\AirBnB Data\Listings gz"

listings_list = []

with os.scandir(gz_listings_path) as entries:
    for entry in entries:
        if entry.is_file():
            listings_list.append(pd.read_csv(entry.path))

listings_df = pd.concat(listings_list)
listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6422,https://www.airbnb.com/rooms/6422,20230622195211,2023-06-22,city scrape,Home in Nashville · ★4.95 · 1 bedroom · 1 bed ...,30 day plus rental - book for one month and th...,Historic East Nashville is home to many new an...,https://a0.muscache.com/pictures/miso/Hosting-...,12172,...,4.96,4.92,4.98,NaN,f,1,0,1,0,3.91
1,39870,https://www.airbnb.com/rooms/39870,20230622195211,2023-06-22,city scrape,Home in Nashville · ★4.94 · 1 bedroom · 1 bed ...,"Since I am older, I need for guests to be vacc...","The house is in a safe, quiet, ""college"" neig...",https://a0.muscache.com/pictures/miso/Hosting-...,171184,...,4.98,4.94,4.94,NaN,f,1,0,1,0,4.52
2,72906,https://www.airbnb.com/rooms/72906,20230622195211,2023-06-23,city scrape,Rental unit in Nashville · ★4.91 · 2 bedrooms ...,Entire top floor. Private. 800 sq ft of bright...,Historic Belmont-Hillsboro neighborhood. Walk-...,https://a0.muscache.com/pictures/58602855/3788...,176117,...,4.99,4.96,4.89,NaN,f,1,1,0,0,4.71
3,258817,https://www.airbnb.com/rooms/258817,20230622195211,2023-06-22,city scrape,Home in Nashville · ★4.77 · 1 bedroom · 2 beds...,Morningstar House! Extended stay - 3+ months- ...,"I'm very near the Percy Priest Lake, natural ...",https://a0.muscache.com/pictures/eef19bc4-2743...,22296,...,4.90,4.74,4.73,NaN,f,8,0,7,1,0.68
4,289242,https://www.airbnb.com/rooms/289242,20230622195211,2023-06-22,city scrape,Home in Nashville · ★4.71 · 2 bedrooms · 3 bed...,Morningstar House! Extended stay - price reduc...,The Morningstar House is in a quiet cul-de-sac...,https://a0.muscache.com/pictures/40a7ab72-20f7...,22296,...,4.78,4.47,4.69,NaN,f,8,0,7,1,0.54


In [29]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228701 entries, 0 to 44463
Data columns (total 75 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   id                                            228701 non-null  int64  
 1   listing_url                                   228701 non-null  object 
 2   scrape_id                                     228701 non-null  int64  
 3   last_scraped                                  228701 non-null  object 
 4   source                                        228701 non-null  object 
 5   name                                          228701 non-null  object 
 6   description                                   226441 non-null  object 
 7   neighborhood_overview                         144375 non-null  object 
 8   picture_url                                   228700 non-null  object 
 9   host_id                                       228701 n

In [ ]:
sql = """
SELECT
    r."index",
    r.id,
    r.comments,
    l."index",
    l.review_scores_rating

    """